# Problem 19: Application of a Groundwater Flow Model to a Water Supply Problem

# INTRODUCTION
_Groundwater flow models are often used in term productivity of local or regional aquifers.
water resource evaluations to assess the longThis exercise presents an example of an application to a local system and involves calibration to an aquifer test and prediction using best estimates of aquifer properties. Of historical interest, this problem is adapted from one of the first applications of a digital model to a water resource problem (Pinder and Bredehoeft, 1968). The specific objective of their study was to assess whether a glaciofluviaI aquifer could provide an adequate water supply for a village in Nova Scotia._


# PROBLEM STATEMENT AND DATA
_The aquifer is located adjacent to the Musquodoboit River, l/4-mile northwest of the village of Musquodoboit Harbour, as shown in Figure 19.1. The aquifer is a glaciofluvial deposit consisting of coarse sand, gravel, cobbles, and boulders deposited in a typical Ushaped glacial valley cut into the slates and quartzites of the Meguma group and the granite intrusive of Devonian age. The contrast in permeability between the granitic and metamorphic rocks and the glaciofluvial valley fill is so great (approximately 106) that the bedrock is considered as impermeable in the aquifer analysis. The aquifer, which is up to 62 feet thick, is extensively overlain by recent alluvial deposits of sand, silt and clay. The alluvial deposits are less permeable and act as confining beds. A cross-section through the valley is given in Figure 19.2._

_A pumping test was conducted to evaluate the aquifer transmissivity and storage coefficient, and to estimate recharge from the river. The test was run for 36 hours using a well discharging at 0.963 cubic feet per second (432 gallons per minute) and three observation wells (see inset of Figure 19.1 for locations). The test was discontinued when the water level in the pumping well became stable. Initial estimates of aquifer parameters were calculated using the Theis curve and the early segment of the drawdown curves for the observation wells. The results were somewhat variable, ranging from 1.15 ft2/s to 1.45 ft2/s. A quasi-steady state formula for estimating transmissivity yielded results on the order of 0.3 ft2/s. Because of the close proximity of boundaries, the pumping test results are difficult to analyze using usual analytical methods._

_A listing of the data set for the MODFLOW model is provided on page 19-4. The aquifer is treated as confined, with transmissivity zones to account for thickness and facies changes. The ratio between zones of transmissivity (1,2, and 4) are given in the data set; absolute values of transmissivity are not given. A map of the transmissivity zones and model boundaries is given in Figure 19.3._

_A uniform value of storage coefficient is used in the analysis. The model is used to simulate drawdown, hence an initial head condition of 0.0 ft is used. Recharge is not specified because only drawdown is simulated. A river is simulated using the RIVER package. Its location is shown in Figure 19.4. Other pertinent data is given in Table 19.1._


## Part c)
_Make some conclusions:_

_How good is your history match?_


_What additional changes might improve it?_


_How important is river leakage?_


_How appropriate is the confined model approximation?_


_How much confidence do you have in you prediction?_


_Is the system at steady-state at 1000 days?


_What are some weaknesses in the calibration/prediction procedure?_


_What does the modeling indicate regarding the feasibility of using this aquifer as a water supply?_


In [1]:
#Import and modules/packages that we need to work with
%matplotlib inline
import os
import sys
import platform
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import flopy

In [2]:
#Store the values of each parameter into separate variables
inh = 0.0 #initial head
hk = .3*86400 #hydraulic conductivity ft^2/day
ss = 0.06 #storage coefficient
delr = 100 #grid spacing (uniform)
delc = delr
pr = -.963/7.48*86400 #pumping rate ft^3/day
perlen = 1.5 #stress period length in days
nstp = 10 #number of time steps
tsmult = 1.414 #time step multiplier
it_par = 5 # iteration parameters
mxiter = 50 #maximum number of iterations
accel_par = 1.0 #acceleration parameter
hclose = 0.001 #closure criterion (the head change criterion for convergence)
top = 0
botm = -1
nper = 1
laytyp = 0
ipakcb = 50

#grid 44 rows, 55 cols, 1 layer
nlay = 1 #number of layers
nrow = 44 #number of rows
ncol = 55 #number of columns

#riv package
rstage = 0.0 #river stage
rcond = 1728 #river conductance in ft^2/day
rbot = -10 #river bottom

#print to double check the math is done correctly
print(hk)
print(pr)

25920.0
-11123.422459893047


In [3]:
#Flopy objects
modelname = 'p19' #short for problem 19
exe_name = '../MF2005.1_12/bin/mf2005.exe'
model_ws = 'temp'
mf = flopy.modflow.Modflow(modelname, exe_name=exe_name, model_ws = model_ws)

#call in the discretization package
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc,
                               top=top, botm=botm, nper=nper, perlen=perlen, 
                               nstp=nstp, tsmult=tsmult, steady=False)

In [6]:
fname='..\Data\p19_trans_double.txt'
print(os.path.isfile(fname))
with open(fname) as f:
    data = f.read().splitlines()
    
f.close()
#print(data)

ibound = []
for s in data:
    for value in s.strip().split(' '):
        ibound.append(int(value))
ibound = np.array(ibound)
ibound = ibound.reshape((44, 55))
plt.imshow(ibound)
hk = np.empty((44, 55), dtype=np.float)
hk[:, :] = ibound[:, :]

hk1 = 35000.
hk2 = 29000.
hk4 = 18000.

hk = np.where(hk==1, hk1, hk)
hk = np.where(hk==2, hk2, hk)
hk = np.where(hk==4, hk4, hk)
plt.imshow(hk)
plt.colorbar()

lpf = flopy.modflow.ModflowLpf(mf, hk=hk, ss=ss, laytyp=laytyp, ipakcb=ipakcb)

True


ValueError: total size of new array must be unchanged

In [7]:
#call in the basic package
strt = 0.0
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)

#call in the well package (WEL)
# Remember to use zero-based layer, row, column indices!
wel_sp1 = [[0, 27, 31, pr], [0, 27, 31, pr], [0, 28, 34, pr], [0, 30, 31, pr]]
wel_spd = {0: wel_sp1}
wel = flopy.modflow.ModflowWel(mf, stress_period_data=wel_spd)

#call in the preconditioned conjugate-gradient package (PCG) 
pcg = flopy.modflow.ModflowPcg(mf, mxiter=mxiter, hclose=hclose)

#call in the output control package (OC)
oc = flopy.modflow.ModflowOc(mf, stress_period_data={(0, 0): ['save head', 'print budget', 'save budget']})

#call in the river package (RIV)
#figure out how to change conductance from the package
riv_boundary = pd.read_csv('../Data/problem19_riv.csv')
riv_boundary.rcond = 1728
riv_boundary.layer -=1
riv_boundary.row -=1
riv_boundary.col -=1
print(riv_boundary.values)

riv_spd = {0: riv_boundary.values}

riv = flopy.modflow.ModflowRiv(mf, stress_period_data=riv_spd, ipakcb=ipakcb)

AssertionError: length of 3d enumerable:2400 != to shape[0]:1

In [8]:
# Write the MODFLOW model input files
mf.write_input()

# Run the MODFLOW model
success, buff = mf.run_model()

FloPy is using the following executable to run the model: ../MF2005.1_12/bin/mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: p19.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2017/08/14 11:05:43

